# NBA Data Science Analysis

**Background** 
<br><br>
This analysis is in accordance with a free weekly Daily Fantasy NBA Basketball competition hosted by Yahoo. <br>I have participated in this competition in the past and decided to utilize the competition to forecast/predict player's performances and optimizing a potential lineup based on those predictions. <br><br> This notebook walks through exploratory data analysis of different factors on player performance, prediction analysis to find the most accurate forecast method, manipulation of the different data sources into a desired format, and an optimization script to produce a lineup to utilize for a given week's competition.

#### Import Needed Packages

In [47]:
import pandas as pd
import numpy as np
from pulp import *
import matplotlib.pyplot as plt
from bokeh.layouts import row, column
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.io import show, output_notebook
from bokeh.palettes import Viridis5, Spectral5, Viridis256
from bokeh.transform import factor_cmap
from bokeh.layouts import gridplot
import statsmodels
dat = '5-16'

##### Summary of Data Being Used

**NBA_PLAYER_BOX_SCORES:** Data from NBA 2020-2021 Season of all individual player box scores
<br>
**Opponent List:** List of team's opponents for each date of NBA fantasy competition
<br>
**Yahoo DFS Cost:** Data of player's to choose from and their associated costs for each competition

In [23]:
file1 = ('https://github.com/ZTFisme/Data-Sets/blob/main/NBA_PLAYER_BOX_SCORES_2021.xlsx?raw=true')
file2 = ('https://github.com/ZTFisme/Data-Sets/blob/main/Opponent_List.xlsx?raw=true')
file3 = ('https://github.com/ZTFisme/Data-Sets/blob/main/Yahoo_DFS_Cost.xlsx?raw=true')

data = pd.read_excel(file1, sheet_name = 0, header=0)
data.describe()

,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,DREB,...,TOV,PF,+/-,FPTS,FPTS/MIN,PTS_DIFF,OPP_DEFRTG,OPP_REB%,OPP_TOV%,OPP_PACE
count,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,...,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.000000,22456.00000
mean,22.693623,10.529035,3.870369,8.305887,1.193757,3.254498,1.594540,2.050143,0.923495,3.239223,...,1.243498,1.814214,-0.000891,21.276011,0.886601,0.176280,111.487384,50.013609,13.888783,99.69756
std,10.511594,8.652074,3.214721,5.931299,1.500419,3.026761,2.234617,2.691348,1.317078,2.736013,...,1.397114,1.438487,11.414117,14.498026,0.445033,9.359054,2.575925,1.633672,1.072825,1.84060
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-50.000000,-3.000000,-1.000000,-48.554717,106.000000,47.200000,11.500000,96.46000
25%,15.000000,4.000000,1.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,-7.000000,10.000000,0.603671,-6.100000,110.000000,48.900000,13.200000,98.27000
50%,24.000000,9.000000,3.000000,7.000000,1.000000,3.000000,1.000000,1.000000,0.000000,3.000000,...,1.000000,2.000000,0.000000,19.400000,0.869631,-0.651627,111.700000,49.500000,14.000000,99.36000
75%,31.000000,15.000000,6.000000,12.000000,2.000000,5.000000,2.000000,3.000000,1.000000,5.000000,...,2.000000,3.000000,7.000000,30.225000,1.150000,5.884332,112.600000,51.400000,14.600000,100.81000
max,51.000000,62.000000,21.000000,37.000000,11.000000,21.000000,19.000000,24.000000,12.000000,20.000000,...,10.000000,6.000000,54.000000,93.900000,5.500000,52.137288,117.900000,52.900000,15.600000,104.74000


In [24]:
data.head()

,PLAYER,TEAM,MATCH UP,GAME DATE,W/L,MIN,PTS,FGM,FGA,FG%,...,H/A,OPP,FPTS/MIN,PTS_DIFF,POSITION,OPP_POS,OPP_DEFRTG,OPP_REB%,OPP_TOV%,OPP_PACE
0,Aaron Gordon,ORL,ORL vs. MIA,2020-12-23,W,26,20,8,11,72.7,...,Home,MIA,1.476923,10.707143,PF,MIA-PF,109.4,48.9,14.8,97.17
1,Aaron Gordon,ORL,ORL @ WAS,2020-12-26,W,30,15,6,12,50,...,Away,WAS,1.010000,2.607143,PF,WAS-PF,112.1,49.0,14.1,104.74
2,Aaron Gordon,ORL,ORL @ WAS,2020-12-27,W,20,4,1,4,25,...,Away,WAS,0.560000,-16.492857,PF,WAS-PF,112.1,49.0,14.1,104.74
3,Aaron Gordon,ORL,ORL @ OKC,2020-12-29,W,22,12,5,10,50,...,Away,OKC,1.090909,-3.692857,PF,OKC-PF,112.6,49.3,15.6,101.18
4,Aaron Gordon,ORL,ORL vs. PHI,2020-12-31,L,21,6,1,8,12.5,...,Home,PHI,0.719048,-12.592857,PF,PHI-PF,107.0,51.8,14.6,100.57


### Exploratory Data Analysis

Look into following packages:
seaborn -- can make plot easily and colorful
altair -- can link plots
bokeh -- interactive version of matplotlib -- can also be used for network graph potentially -- interactivity between plots -- colorful plots >>> https://www.kdnuggets.com/2017/03/bokeh-cheat-sheet.html 

In [25]:
group_by = data.groupby("POSITION")["FPTS/MIN","FPTS"].mean()
source = ColumnDataSource(group_by)
positions = source.data['POSITION'].tolist()

p = figure(x_range=positions)
color_map = factor_cmap(field_name='POSITION', palette=Viridis5, factors=positions)

p.vbar(x='POSITION', top='FPTS/MIN', source=source, width=0.50, color=color_map)
p.title.text ='Average Fantasy Points per Minute by Position'
p.xaxis.axis_label = 'Position'
p.yaxis.axis_label = 'Average Fantasy Points per Minute per Game'


source2 = ColumnDataSource(group_by)
positions = source2.data['POSITION'].tolist()

p2 = figure(x_range=positions)
color_map = factor_cmap(field_name='POSITION', palette=Viridis5, factors=positions)
p2.vbar(x='POSITION', top='FPTS', source=source, width=0.50, color=color_map)

p2.title.text ='Average Fantasy Points by Position'
p2.xaxis.axis_label = 'Position'
p2.yaxis.axis_label = 'Average Fantasy Points per Game'

output_notebook()
show(column(p,p2))

Loading BokehJS ...

In [52]:
pg_data = data.where(data['POSITION']=='PG')
sg_data = data.where(data['POSITION']=='SG')
sf_data = data.where(data['POSITION']=='SF')
pf_data = data.where(data['POSITION']=='PF')
c_data = data.where(data['POSITION']=='C')

pg_group_by = pg_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='FPTS')
sg_group_by = sg_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='FPTS')
sf_group_by = sf_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='FPTS')
pf_group_by = pf_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='FPTS')
c_group_by = c_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='FPTS')

pg_source = ColumnDataSource(pg_group_by)
pg_opponents = pg_source.data['OPP'].tolist()
pg = figure(x_range = pg_opponents, sizing_mode='scale_width')
pg_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = pg_opponents)
pg.vbar(x='OPP', top='FPTS', source = pg_source, width=0.45, color = pg_color_map)
pg.title.text ='PG'
pg.xaxis.axis_label = 'Opponent'
pg.yaxis.axis_label = 'PG Average Fantasy Points per Game'
pg.xaxis.major_label_orientation = "vertical"

sg_source = ColumnDataSource(sg_group_by)
sg_opponents = sg_source.data['OPP'].tolist()
sg = figure(x_range = sg_opponents, sizing_mode='scale_width')
sg_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = sg_opponents)
sg.vbar(x='OPP', top='FPTS', source = sg_source, width=0.45, color = sg_color_map)
sg.title.text ='SG'
sg.xaxis.axis_label = 'Opponent'
sg.yaxis.axis_label = 'SG Average Fantasy Points per Game'
sg.xaxis.major_label_orientation = "vertical"

sf_source = ColumnDataSource(sf_group_by)
sf_opponents = sf_source.data['OPP'].tolist()
sf = figure(x_range = sf_opponents, sizing_mode='scale_width')
sf_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = sf_opponents)
sf.vbar(x='OPP', top='FPTS', source = sf_source, width=0.45, color = sf_color_map)
sf.title.text ='SF'
sf.xaxis.axis_label = 'Opponent'
sf.yaxis.axis_label = 'SF Average Fantasy Points per Game'
sf.xaxis.major_label_orientation = "vertical"

pf_source = ColumnDataSource(pf_group_by)
pf_opponents = pf_source.data['OPP'].tolist()
pf = figure(x_range = pf_opponents, sizing_mode='scale_width')
pf_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = pf_opponents)
pf.vbar(x='OPP', top='FPTS', source = pf_source, width=0.45, color = pf_color_map)
pf.title.text ='PF'
pf.xaxis.axis_label = 'Opponent'
pf.yaxis.axis_label = 'PF Average Fantasy Points per Game'
pf.xaxis.major_label_orientation = "vertical"

c_source = ColumnDataSource(c_group_by)
c_opponents = c_source.data['OPP'].tolist()
c = figure(x_range = c_opponents, sizing_mode='scale_width')
c_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = c_opponents)
c.vbar(x='OPP', top='FPTS', source = c_source, width=0.45, color = c_color_map)
c.title.text ='C'
c.xaxis.axis_label = 'Opponent'
c.yaxis.axis_label = 'C Average Fantasy Points per Game'
c.xaxis.major_label_orientation = "vertical"

output_notebook()
grid = gridplot([pg, sg, sf, pf, c], ncols=3, plot_width=450, plot_height=350)
show(grid)

Loading BokehJS ...

In [57]:
pg_group_by = pg_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='PTS_DIFF')
sg_group_by = sg_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='PTS_DIFF')
sf_group_by = sf_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='PTS_DIFF')
pf_group_by = pf_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='PTS_DIFF')
c_group_by = c_data.groupby("OPP")["PTS_DIFF", 'FPTS'].mean().sort_values(ascending = False, by ='PTS_DIFF')

pg_source = ColumnDataSource(pg_group_by)
pg_opponents = pg_source.data['OPP'].tolist()
pg = figure(x_range = pg_opponents, sizing_mode='scale_width')
pg_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = pg_opponents)
pg.vbar(x='OPP', top='PTS_DIFF', source = pg_source, width=0.45, color = pg_color_map)
pg.title.text ='PG'
pg.xaxis.axis_label = 'Opponent'
pg.yaxis.axis_label = 'PTS DIFF'
pg.xaxis.major_label_orientation = "vertical"

sg_source = ColumnDataSource(sg_group_by)
sg_opponents = sg_source.data['OPP'].tolist()
sg = figure(x_range = sg_opponents, sizing_mode='scale_width')
sg_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = sg_opponents)
sg.vbar(x='OPP', top='PTS_DIFF', source = sg_source, width=0.45, color = sg_color_map)
sg.title.text ='SG'
sg.xaxis.axis_label = 'Opponent'
sg.yaxis.axis_label = 'PTS DIFF'
sg.xaxis.major_label_orientation = "vertical"

sf_source = ColumnDataSource(sf_group_by)
sf_opponents = sf_source.data['OPP'].tolist()
sf = figure(x_range = sf_opponents, sizing_mode='scale_width')
sf_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = sf_opponents)
sf.vbar(x='OPP', top='PTS_DIFF', source = sf_source, width=0.45, color = sf_color_map)
sf.title.text ='SF'
sf.xaxis.axis_label = 'Opponent'
sf.yaxis.axis_label = 'PTS DIFF'
sf.xaxis.major_label_orientation = "vertical"

pf_source = ColumnDataSource(pf_group_by)
pf_opponents = pf_source.data['OPP'].tolist()
pf = figure(x_range = pf_opponents, sizing_mode='scale_width')
pf_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = pf_opponents)
pf.vbar(x='OPP', top='PTS_DIFF', source = pf_source, width=0.45, color = pf_color_map)
pf.title.text ='PF'
pf.xaxis.axis_label = 'Opponent'
pf.yaxis.axis_label = 'PTS DIFF'
pf.xaxis.major_label_orientation = "vertical"

c_source = ColumnDataSource(c_group_by)
c_opponents = c_source.data['OPP'].tolist()
c = figure(x_range = c_opponents, sizing_mode='scale_width')
c_color_map = factor_cmap(field_name = 'OPP', palette = Viridis256, factors = c_opponents)
c.vbar(x='OPP', top='PTS_DIFF', source = c_source, width=0.45, color = c_color_map)
c.title.text ='C'
c.xaxis.axis_label = 'Opponent'
c.yaxis.axis_label = 'PTS DIFF'
c.xaxis.major_label_orientation = "vertical"

output_notebook()
grid = gridplot([pg, sg, sf, pf, c], ncols=3, plot_width=450, plot_height=350)
show(grid)

Loading BokehJS ...

## Forecasting Analysis


### Moving Averages: Simple and Exponential

In [ ]:
tab = pd.DataFrame(columns = ['Type','K-Value', 'MAD', 'MSD', 'MAPE'])

for k in range(2,11):
    forecast = data.groupby('PLAYER').rolling(k)['FPTS'].mean().reset_index(drop=True)
    MAD = (forecast - data['FPTS']).abs()
    MSD = (forecast - data['FPTS']).abs().pow(2)
    MAPE = (((data['FPTS']- forecast)/data['FPTS']).abs())
    tab.at[k-1, 'Type'] = 'SMA'
    tab.at[k-1, 'K-Value'] = k
    tab.at[k-1, 'MAD'] = MAD.mean()
    tab.at[k-1, 'MSD'] = MSD.mean()
    tab.at[k-1, 'MAPE'] = MAPE.median()
    
    ema = (data.groupby(['PLAYER'])['FPTS'].transform(lambda x: x.ewm(span=k).mean()))
    ema_MAD = (ema - data['FPTS']).abs()
    ema_MSD = (ema - data['FPTS']).abs().pow(2)
    ema_MAPE = (((data['FPTS']- ema)/data['FPTS']).abs())
    tab.at[k+9, 'Type'] = 'EMA'
    tab.at[k+9, 'K-Value'] = k
    tab.at[k+9, 'MAD'] = ema_MAD.mean()
    tab.at[k+9, 'MSD'] = ema_MSD.mean()
    tab.at[k+9, 'MAPE'] = ema_MAPE.median()
print ('\033[1m' + 'Moving Average Forecasts:\n')
tab.sort_values(by='MAD',ascending=True).reset_index(drop=True)

Utilize EMA with k-value of 2 since it is most accurate.

In [ ]:
data['EMA2'] = ((data.groupby(['PLAYER'])['FPTS'].transform(lambda x: x.ewm(span=2).mean())))

In [ ]:
opp_pos = data.groupby(['OPP','POSITION','OPP_POS'])['PTS_DIFF','FPTS', 'FPTS/MIN'].mean().reset_index()
opp_c = opp_pos[opp_pos['POSITION']=='C'].reset_index(drop=True)
opp_pf = opp_pos[opp_pos['POSITION']=='PF'].reset_index(drop=True)
opp_sf = opp_pos[opp_pos['POSITION']=='SF'].reset_index(drop=True)
opp_sg = opp_pos[opp_pos['POSITION']=='SG'].reset_index(drop=True)
opp_pg = opp_pos[opp_pos['POSITION']=='PG'].reset_index(drop=True)

In [ ]:
def opp_process(opp):
    fpts_mu = opp['FPTS'].mean()
    diff_mu = opp['PTS_DIFF'].mean()
    fpts_min_mu = opp['FPTS/MIN'].mean()
    
    opp['FPTS_AGST_AVG'] = ''
    opp['DIFF_AGST_AVG'] = ''
    opp['FPTS_MIN_AGST_AVG'] = ''
    
    for i in range(len(opp_pg)):
        fpt = opp_pg.iloc[i]['FPTS'] / fpts_mu
        diff = opp_pg.iloc[i]['PTS_DIFF'] - diff_mu
        fpts_min = opp_pg.iloc[i]['FPTS/MIN'] / fpts_min_mu
        opp.at[i, 'FPTS_AGST_AVG'] = fpt
        opp.at[i, 'DIFF_AGST_AVG'] = diff
        opp.at[i, 'FPTS_MIN_AGST_AVG'] = fpts_min
    return opp

In [ ]:
opp_pg = opp_process(opp_pg)
opp_sg = opp_process(opp_sg)
opp_sf = opp_process(opp_sf)
opp_pf = opp_process(opp_pf)
opp_c = opp_process(opp_c)

In [ ]:
opps = [opp_pg, opp_sg, opp_sf, opp_pf, opp_c]
opponents = pd.concat(opps).reset_index(drop=True)
opponents = opponents[['OPP_POS','FPTS_AGST_AVG','PTS_DIFF','FPTS_MIN_AGST_AVG']]
#opponents = opponents[['OPP_POS','FPTS_MIN_AGST_AVG']]

In [ ]:
player_stats = pd.DataFrame(data.groupby('PLAYER')['FPTS','FPTS/MIN'].mean())

In [ ]:
last = pd.DataFrame(data.set_index('PLAYER').groupby(level='PLAYER').agg(['last']).stack())
last = last.reset_index()[['PLAYER','POSITION','TEAM','EMA2']]
l = last.merge(player_stats, how='inner', left_on = 'PLAYER', right_on = 'PLAYER')

In [ ]:
opponent = pd.read_excel(file2, sheet_name = 0, header=0)
opponent = opponent[['TEAM',dat]].rename(columns={dat:'OPP'})

In [ ]:
oppo = l.merge(opponent, how='inner', left_on = 'TEAM', right_on = 'TEAM')
oppo['OPP_POS'] = oppo['OPP'] + '-' + oppo['POSITION']
pred = oppo.merge(opponents, how = 'inner', left_on='OPP_POS', right_on = 'OPP_POS')
pred = pred.rename(columns={'OPP_x':'OPP','POSITION_x':'POSITION'})
pred['PRED'] = ((((pred['EMA2']*0.4) + (pred['FPTS'])*0.6)) + pred['PTS_DIFF'])
cost = pd.read_excel(file3, sheet_name = dat, header=0)
cost = cost[['PLAYER', 'COST']]
pred = pred.merge(cost, how = 'inner', left_on = 'PLAYER', right_on = 'PLAYER')
pred.head()

In [ ]:
df = pred
def pg_id(x):
    if x=='PG':
        return 1
    else:
        return 0
def sg_id(x):
    if x=='SG':
        return 1
    else:
        return 0
def sf_id(x):
    if x=='SF':
        return 1
    else:
        return 0
def pf_id(x):
    if x=='PF':
        return 1
    else:
        return 0
def c_id(x):
    if x=='C':
        return 1
    else:
        return 0
    
#One hot encoder for position
#label_encoder = LabelEncoder()
#df['NEW_POS'] = label_encoder.fit_transform(df['Position'])

df['PG'] = df['POSITION'].apply(pg_id)
df['SG'] = df['POSITION'].apply(sg_id)
df['SF'] = df['POSITION'].apply(sf_id)
df['PF'] = df['POSITION'].apply(pf_id)
df['C'] = df['POSITION'].apply(c_id)
df['COST'] = [float(i) for i in df['COST']]

#Clean data and convert to list- only look at rows with diet data and not constraint information
df = df.where(df['COST'] > 0)
df = df.dropna()
df = df.values.tolist()

players = [x[0] for x in df]
cost = dict([(x[0], float(x[12])) for x in df])
position = dict([(x[0], x[1]) for x in df])
proj_pts = dict([(x[0], float(x[11])) for x in df])

pg = dict([(x[0], float(x[13])) for x in df])
sg = dict([(x[0], float(x[14])) for x in df])
sf = dict([(x[0], float(x[15])) for x in df])
pf = dict([(x[0], float(x[16])) for x in df])
c = dict([(x[0], float(x[17])) for x in df])

player_vars = LpVariable.dicts("Player", players, cat = "Integer", lowBound= 0, upBound = 1)

# create the optimization problem framework - maximize points while meeting requirements of maximum cost
prob = LpProblem("NBA_Optimize", LpMaximize)

#Define objective function for projected points
obj_func = lpSum([proj_pts[i] * player_vars[i] for i in players])
prob += obj_func

#Number of player choices constraint
constraint_2 = lpSum([player_vars[i] for i in player_vars]) == 8
prob += constraint_2 
 
#Cost Constraint
constraint_3 = lpSum([cost[f] * player_vars[f] for f in player_vars]) <= 200.0
prob += constraint_3

#Position Constraints
#PG
prob += lpSum([pg[f] * player_vars[f] for f in player_vars]) >= 1
prob += lpSum([pg[f] * player_vars[f] for f in player_vars]) <= 3

#SG
prob += lpSum([sg[f] * player_vars[f] for f in player_vars]) >= 1
prob += lpSum([sg[f] * player_vars[f] for f in player_vars]) <= 3

#G
prob += lpSum([(pg[f] * player_vars[f]) + (sg[f] * player_vars[f]) for f in player_vars]) >= 3
prob += lpSum([(pg[f] * player_vars[f]) + (sg[f] * player_vars[f]) for f in player_vars]) <= 4

#SF
prob += lpSum([sf[f] * player_vars[f] for f in player_vars]) >= 1
prob += lpSum([sf[f] * player_vars[f] for f in player_vars]) <= 3

#PF
prob += lpSum([pf[f] * player_vars[f] for f in player_vars]) >= 1
prob += lpSum([pf[f] * player_vars[f] for f in player_vars]) <= 3

#F
prob += lpSum([(sf[f] * player_vars[f]) + (pf[f] * player_vars[f]) for f in player_vars]) >= 3
prob += lpSum([(sf[f] * player_vars[f]) + (pf[f] * player_vars[f]) for f in player_vars]) <= 4

#C
prob += lpSum([c[f] * player_vars[f] for f in player_vars]) >= 1
prob += lpSum([c[f] * player_vars[f] for f in player_vars]) <= 2

#Solve the Objective Function
prob.solve()
name_lst = []
import re
for x in prob.variables():
    if x.varValue>0:
        nm = x.name
        name_lst.append(re.sub('_',' ',nm[7:]))
lineup = pd.DataFrame(name_lst).rename(columns={0:'PLAYER'})
lineup = lineup.merge(pred, how = 'inner', left_on = 'PLAYER', right_on = 'PLAYER')
lineup = lineup[['PLAYER', 'POSITION', 'EMA2', 'FPTS', 'FPTS/MIN', 'COST', 'PRED']]
print('Optimal Lineup is: \n\n', lineup, '\n\n\nThe Projected Total Points are: ', sum(lineup['PRED']))